# 02: H-Atom Fast Forward

Recent literature has shown methods such as Variational Fast Forwarding which aims to predict the long-term time evolutions of a system given data on the short-term evolutions of the system. Filip et al [1] and Cîrstoiu et al [2] have demonstrated two different ways to train a VFF using short-term evolution data. This dataset provides us short-term data and gives us testing points for long-term performance benchmarking

## 02.1 Data Format

The data generator gives us the following:
- `⎢ѱHK〉`: The Hartree Fock State of the system
- `X_train`: `[Δt, 2Δtm …MΔt]` which are uniform timestamps at which short-term evolutions are recorded
- `Y_train`: `U(Δt)⎢ѱHK〉...U(MΔt)⎢ѱHK〉` which are the short-term evolutions but with noise
- `X_test`: `[PΔt…NΔt]` which are the long-term timestamps
- `Y_test`: `U(PΔt)⎢ѱHK〉…U(NΔt)⎢ѱHK〉` which are the long-term noiseless evolutions

The data generator gets the `U(Δt)` and the `⎢ѱHK〉` from a H-atom model given by the hyperparameters. The noisy generations allow us to also benchmark our pipeline on how well it avoids overfitting to noise which wasn't discussed in the references above

### 02.1.1 H-Atom Hamiltonians

We need some Chemistry to first decide what geometries to import.

- H2: Usually found at a `0.735 Å` equilibrium bond distance [O’Malley et al. (2016)]
- H3+: Usually found both in an equilateral triangle configuration and a linear configuration. We will proceed with the linear one with `0.9 Å` between each pair
- H6: Usually modelled as a linear chain of 6 atoms with bond lengths `1 Å` between each pair


#### Electronic Hamiltonians from PySCF 

ImportError: attempted relative import with no known parent package

In [ ]:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

num_alpha_electrons = es_problem.num_alpha 
num_beta_electrons = es_problem.num_beta 
num_spin_orbitals = int(es_problem.num_spin_orbitals) 

nuclear_rep_energy = es_problem.nuclear_repulsion_energy 
print("number of alpha electrons: " , num_alpha_electrons)
print("number of beta electrons: " , num_beta_electrons)
print("number of spin orbitals: " , num_spin_orbitals)
print("nuclear repulsion energy: " , nuclear_rep_energy)

hamiltonian = es_problem.hamiltonian 
second_q_op = hamiltonian.second_q_op() 
mapper = JordanWignerMapper()
qubit_op = mapper.map(second_q_op) 

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend('ibm_sherbrooke')
# backend = service.least_busy(simulator=False)
# estimator = Estimator(backend_options={"method": "statevector"})
estimator = EstimatorV2(backend)

# from qiskit.utils import algorithm_globals
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.circuit.library import HartreeFock

# algorithm_globals.random_seed = 1024
convergence = []
job_list = []


def evaluate_expectation(x):
    x = list(x)
    job = estimator.run(circuits=[ansatz], observables=[qubit_op_parity], parameter_values=[x]).result()

    results = job.values[0]
    job_list.append(results)
    convergence.append(np.real(results))
    return np.real(results)

np.random.seed(10)

init_state = HartreeFock(num_spatial_orbitals=es_problem.num_spatial_orbitals, num_particles= es_problem.num_particles, qubit_mapper=mapper) 

ansatz = UCC(num_spatial_orbitals=es_problem.num_spatial_orbitals, num_particles=es_problem.num_particles, excitations=[1, 2], qubit_mapper=mapper, initial_state = init_state) 

initial_point = np.random.random(ansatz.num_parameters)
optimizer = SPSA(maxiter = 50) 

result = optimizer.minimize(evaluate_expectation,x0= initial_point)

### 02.1.1 Hamiltonian Models
